<a href="https://colab.research.google.com/github/Gus-1003/ISD-invention/blob/main/Pesquisa/3_laborat%C3%B3rio_de_varia%C3%A7%C3%B5es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bibliotecas:

In [ ]:
import os

import glob

import cv2

import numpy as np

import pandas as pd

from google.colab.patches import cv2_imshow

from google.colab import drive

drive.mount('/content/drive/')
%cd /content/drive/MyDrive/

Mounted at /content/drive/
/content/drive/MyDrive


# Imagem Padrão:

In [ ]:
frame1 = '/content/drive/MyDrive/Pesquisa_ISD_IC/Imagens_Infravermelho/Dia4-TR-CCF504-Contexto4_frame0.jpg'

# Carrega a imagem especificada pelo caminho do arquivo.
frame1 = cv2.imread(frame1)
# Exibe a imagem pos-filtro.
cv2_imshow(frame1)

# Algoritmo para remoção do Rato:

In [ ]:
frame1 = '/content/drive/MyDrive/Pesquisa_ISD_IC/Imagens_Infravermelho/Dia4-TR-CCF504-Contexto4_frame0.jpg'
frame2 = '/content/drive/MyDrive/Pesquisa_ISD_IC/Imagens_Infravermelho/Dia4-TR-CCF508-Contexto4_frame2.jpg'
frame3 = '/content/drive/MyDrive/Pesquisa_ISD_IC/Imagens_Infravermelho/Dia4-TR-CCF504-Contexto4_frame2.jpg'


# Carrega a imagem especificada pelo caminho do arquivo.
im1 = cv2.imread(frame1)
im2 = cv2.imread(frame2)
im3 = cv2.imread(frame3)

# Criando uma lista com as três imagens
images = [im1, im2, im3]

# Convertendo as imagens em formato de array numpy
images_array = np.array(images)

# Calculando a mediana ao longo do eixo temporal (eixo 0)
median_image = np.median(images_array, axis=0)

# Convertendo a imagem resultante de volta para o tipo de dados uint8
im = median_image.astype('uint8')

# Exibe a imagem pos-filtro.
cv2_imshow(im)

# Diferentes Processos:

## Fase 1: Adição dos filtros gaussiano e Laplaciano

In [ ]:
# Cria uma cópia da imagem original.
img_contours = im.copy()

# Converte a imagem para o espaço de cores em escala de cinza.
image = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

# Converte a imagem para o tipo float32.
image32f = np.float32(image)


# =====================================================================

# Aplica um filtro de média à imagem.
mu = cv2.blur(image32f, (3, 3))

# Aplica um filtro de variância à imagem.
mu2 = cv2.blur(image32f*image32f, (3, 3))
sigma = cv2.sqrt(mu2 - mu*mu)
sigma = sigma * 10
sigma = sigma.astype("uint8")

# Aplica um filtro de borramento com kernel de 5x5
sigma = cv2.GaussianBlur(sigma, (5, 5), 0)

# Aplicando o filtro Laplaciano
laplacian = cv2.Laplacian(sigma, cv2.CV_32F)

# =====================================================================

# Normalizando a saída para o intervalo de 0 a 255
sigma = cv2.convertScaleAbs(laplacian)

# Aplica uma limiarização com método de Otsu à imagem.
ret3,th3 = cv2.threshold(sigma,0,255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# =====================================================================

# Define o kernel para dilatação e erosão
kernel = np.ones((5,5), np.uint8)

# Dilata a imagem
img_dilated = cv2.dilate(th3, kernel, iterations=2)

# Erode a imagem
th3 = cv2.erode(img_dilated, kernel, iterations=2)

# =====================================================================

# Encontra os contornos na imagem limiarizada.
contours, hierarchy = cv2.findContours(th3, cv2.RETR_TREE, cv2.ADAPTIVE_THRESH_GAUSSIAN_C) # https://doi.org/10.1016/0734-189X(85)90016-7

# =====================================================================

# Loop sobre os contornos
for cnt in contours:

  # Aproxima o contorno por uma sequência de pontos
  approx = cv2.approxPolyDP(cnt, 0.01 * cv2.arcLength(cnt, True), True)

  # Calcula a área e o perímetro do contorno
  area = cv2.contourArea(cnt)
  perimeter = cv2.arcLength(cnt, True)

  # Verifica se o contorno é um círculo
  if perimeter > 0:
    circularity = 4 * np.pi * area / (perimeter ** 2)
  else:
    circularity = 0

  # Desenha um círculo
  if circularity > 0.39:

    forma = "circulo"

    (x, y), radius = cv2.minEnclosingCircle(cnt)

    # Limita o raio máximo do círculo
    max_radius = 100  # Define o raio máximo desejado
    radius = min(radius, max_radius)  # Limita o raio ao valor máximo

    center = (int(x), int(y))
    radius = int(radius)

    #if area > 2000 or circularity > 53:
    if area > 2000:
      if radius < 100 and radius > 20:
        cv2.circle(img_contours, center, radius, (255, 0, 0), 2)
        print("================================================================================================================================================================")
        print("================================================================================================================================================================") 
        print(" -> circularity (Circulo): " + str(circularity)  + " / Area (Circulo): " + str(area)  + " / aspect_ratio (Circulo): " + str(aspect_ratio)  + " / radius (Circulo): " + str(radius))
        print("================================================================================================================================================================")
        print("================================================================================================================================================================") 
    
    else:
      print("circularity (Circulo): " + str(circularity)  + " / Area (Circulo): " + str(area)  + " / aspect_ratio (Circulo): " + str(aspect_ratio)  + " / radius (Circulo): " + str(radius))

  # Desenha um quadrado
  else:
    rect = cv2.minAreaRect(cnt)
    
    forma = "quadrado"
    radius = 0

    (x, y), (w, h), angle = rect

    if min(w, h) == 0:
      aspect_ratio = 0
    else:
      aspect_ratio = max(w, h) / min(w, h)
    

    if aspect_ratio < 2.1:
    #if aspect_ratio < 3:  
      # Desenha um quadrado
      box = cv2.boxPoints(rect)
      box = np.int0(box)

      if area > 65:
        cv2.drawContours(img_contours, [box], 0, (0, 0, 255), 2)
        print("================================================================================================================================================================")
        print("================================================================================================================================================================") 
        print(" -> circularity (Quadrado): " + str(circularity)  + " / Area (Quadrado): " + str(area)  + " / aspect_ratio (Quadrado): " + str(aspect_ratio)  + " / radius (Quadrado): " + str(radius))
        print("================================================================================================================================================================")
        print("================================================================================================================================================================") 

    else:
        print("circularity (Quadrado): " + str(circularity)  + " / Area (Quadrado): " + str(area)  + " / aspect_ratio (Quadrado): " + str(aspect_ratio)  + " / radius (Quadrado): " + str(radius))

print()
print()
# Exibe a imagem pos-filtro.
cv2_imshow(mu)
# Exibe a imagem limiarizada.
cv2_imshow(th3)
# Exibe a imagem original copiada com os contornos encontrados.
cv2_imshow(img_contours)
print()
print()
print()

## Fase 2: Adição do filtro de Solber e o calculo de Magnitude do Gradiente

In [ ]:
# Cria uma cópia da imagem original.
img_contours = im.copy()

# Converte a imagem para o espaço de cores em escala de cinza.
image = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

# Converte a imagem para o tipo float32.
image32f = np.float32(image)

# =====================================================================

# Aplica um filtro de média à imagem.
mu = cv2.blur(image32f, (3, 3))

# Aplica um filtro de variância à imagem.
mu2 = cv2.blur(image32f*image32f, (3, 3))
sigma = cv2.sqrt(mu2 - mu*mu)
sigma = sigma * 10
sigma = sigma.astype("uint8")

# Aplica um filtro de borramento com kernel de 5x5
sigma = cv2.GaussianBlur(sigma, (5, 5), 0)

# Aplicando o filtro Laplaciano
laplacian = cv2.Laplacian(sigma, cv2.CV_32F)

# Normalizando a saída para o intervalo de 0 a 255
sigma = cv2.convertScaleAbs(laplacian)

# Aplica um filtro de Sobel em x e y
sobelx = cv2.Sobel(sigma, cv2.CV_64F, 1, 0, ksize=3)
sobely = cv2.Sobel(sigma, cv2.CV_64F, 0, 1, ksize=3)

# =====================================================================

# Calcula a magnitude do gradiente
gradient_magnitude = cv2.magnitude(sobelx, sobely)

# Normaliza a magnitude do gradiente para o intervalo de 0 a 255
gradient_magnitude = cv2.normalize(gradient_magnitude, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

# =====================================================================

# Define o kernel para dilatação e erosão
kernel = np.ones((5,5), np.uint8)

# Aplica uma limiarização com método de Otsu à imagem do gradiente
_, gradient_threshold = cv2.threshold(gradient_magnitude, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# =====================================================================

# Dilata a imagem limiarizada
gradient_threshold = cv2.dilate(gradient_threshold, kernel, iterations=2)

# =====================================================================

# Encontra os contornos na imagem limiarizada.
contours, hierarchy = cv2.findContours(gradient_threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# =====================================================================

# Loop sobre os contornos
for cnt in contours:

  # Aproxima o contorno por uma sequência de pontos
  approx = cv2.approxPolyDP(cnt, 0.01 * cv2.arcLength(cnt, True), True)

  # Calcula a área e o perímetro do contorno
  area = cv2.contourArea(cnt)
  perimeter = cv2.arcLength(cnt, True)

  # Verifica se o contorno é um círculo
  if perimeter > 0:
    circularity = 4 * np.pi * area / (perimeter ** 2)
  else:
    circularity = 0

  # Desenha um círculo
  if circularity > 0.39:

    forma = "circulo"

    (x, y), radius = cv2.minEnclosingCircle(cnt)

    # Limita o raio máximo do círculo
    max_radius = 100  # Define o raio máximo desejado
    radius = min(radius, max_radius)  # Limita o raio ao valor máximo

    center = (int(x), int(y))
    radius = int(radius)

    #if area > 2000 or circularity > 53:
    if area > 2000:
      if radius < 100 and radius > 20:
        cv2.circle(img_contours, center, radius, (255, 0, 0), 2)
        print("================================================================================================================================================================")
        print("================================================================================================================================================================") 
        print(" -> circularity (Circulo): " + str(circularity)  + " / Area (Circulo): " + str(area)  + " / aspect_ratio (Circulo): " + str(aspect_ratio)  + " / radius (Circulo): " + str(radius))
        print("================================================================================================================================================================")
        print("================================================================================================================================================================") 
    
    else:
      print("circularity (Circulo): " + str(circularity)  + " / Area (Circulo): " + str(area)  + " / aspect_ratio (Circulo): " + str(aspect_ratio)  + " / radius (Circulo): " + str(radius))

  # Desenha um quadrado
  else:
    rect = cv2.minAreaRect(cnt)
    
    forma = "quadrado"
    radius = 0

    (x, y), (w, h), angle = rect

    if min(w, h) == 0:
      aspect_ratio = 0
    else:
      aspect_ratio = max(w, h) / min(w, h)
    

    if aspect_ratio < 2.1:
    #if aspect_ratio < 3:  
      # Desenha um quadrado
      box = cv2.boxPoints(rect)
      box = np.int0(box)

      if area > 65:
        cv2.drawContours(img_contours, [box], 0, (0, 0, 255), 2)
        print("================================================================================================================================================================")
        print("================================================================================================================================================================") 
        print(" -> circularity (Quadrado): " + str(circularity)  + " / Area (Quadrado): " + str(area)  + " / aspect_ratio (Quadrado): " + str(aspect_ratio)  + " / radius (Quadrado): " + str(radius))
        print("================================================================================================================================================================")
        print("================================================================================================================================================================") 

    else:
        print("circularity (Quadrado): " + str(circularity)  + " / Area (Quadrado): " + str(area)  + " / aspect_ratio (Quadrado): " + str(aspect_ratio)  + " / radius (Quadrado): " + str(radius))

print()
print()
# Exibe a imagem pos-filtro.
cv2_imshow(mu)
# Exibe a imagem limiarizada.
cv2_imshow(th3)
# Exibe a imagem original copiada com os contornos encontrados.
cv2_imshow(img_contours)
print()
print()
print()

## Fase 3: Adição do Contador de Objetos e limiar max da área dos quadrados com uma diminuição no limiar minimo para circularidade

In [ ]:
# Contador de objetos identificados
objetos = 0

# =====================================================================

# Cria uma cópia da imagem original.
img_contours = im.copy()

# Converte a imagem para o espaço de cores em escala de cinza.
image = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

# Converte a imagem para o tipo float32.
image32f = np.float32(image)

# =====================================================================

# Aplica um filtro de média à imagem.
mu = cv2.blur(image32f, (3, 3))

# Aplica um filtro de variância à imagem.
mu2 = cv2.blur(image32f*image32f, (3, 3))
sigma = cv2.sqrt(mu2 - mu*mu)
sigma = sigma * 10
sigma = sigma.astype("uint8")

# Aplica um filtro de borramento com kernel de 5x5
sigma = cv2.GaussianBlur(sigma, (5, 5), 0)

# Aplicando o filtro Laplaciano
laplacian = cv2.Laplacian(sigma, cv2.CV_32F)

# Normalizando a saída para o intervalo de 0 a 255
sigma = cv2.convertScaleAbs(laplacian)

# Aplica um filtro de Sobel em x e y
sobelx = cv2.Sobel(sigma, cv2.CV_64F, 1, 0, ksize=3)
sobely = cv2.Sobel(sigma, cv2.CV_64F, 0, 1, ksize=3)

# =====================================================================

# Calcula a magnitude do gradiente
gradient_magnitude = cv2.magnitude(sobelx, sobely)

# Normaliza a magnitude do gradiente para o intervalo de 0 a 255
gradient_magnitude = cv2.normalize(gradient_magnitude, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

# =====================================================================

# Aplica uma limiarização com método de Otsu à imagem do gradiente
_, gradient_threshold = cv2.threshold(gradient_magnitude, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# =====================================================================

# Define o kernel para dilatação e erosão
kernel = np.ones((5,5), np.uint8)

# Dilata a imagem limiarizada
gradient_threshold = cv2.dilate(gradient_threshold, kernel, iterations=2)

# =====================================================================

# Encontra os contornos na imagem limiarizada.
contours, hierarchy = cv2.findContours(gradient_threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# =====================================================================

# Loop sobre os contornos
for cnt in contours:

  # Aproxima o contorno por uma sequência de pontos
  approx = cv2.approxPolyDP(cnt, 0.01 * cv2.arcLength(cnt, True), True)

  # Calcula a área e o perímetro do contorno
  area = cv2.contourArea(cnt)
  perimeter = cv2.arcLength(cnt, True)

  # Verifica se o contorno é um círculo
  if perimeter > 0:
    circularity = 4 * np.pi * area / (perimeter ** 2)
  else:
    circularity = 0

  # Desenha um círculo
  if circularity > 0.30:

    forma = "circulo"

    (x, y), radius = cv2.minEnclosingCircle(cnt)

    # Limita o raio máximo do círculo
    max_radius = 100  # Define o raio máximo desejado
    radius = min(radius, max_radius)  # Limita o raio ao valor máximo

    center = (int(x), int(y))
    radius = int(radius)

    #if area > 2000 or circularity > 53:
    if area > 2000:
      if radius < 100 and radius > 20:
        cv2.circle(img_contours, center, radius, (255, 0, 0), 2)
        print("================================================================================================================================================================")
        print("================================================================================================================================================================") 
        print(" -> circularity (Circulo): " + str(circularity)  + " / Area (Circulo): " + str(area)  + " / aspect_ratio (Circulo): " + str(aspect_ratio)  + " / radius (Circulo): " + str(radius))
        print("================================================================================================================================================================")
        print("================================================================================================================================================================") 
        objetos = objetos + 1
    else:
      print("circularity (Circulo): " + str(circularity)  + " / Area (Circulo): " + str(area)  + " / aspect_ratio (Circulo): " + str(aspect_ratio)  + " / radius (Circulo): " + str(radius))

  # Desenha um quadrado
  else:
    rect = cv2.minAreaRect(cnt)
    
    forma = "quadrado"
    radius = 0

    (x, y), (w, h), angle = rect

    if min(w, h) == 0:
      aspect_ratio = 0
    else:
      aspect_ratio = max(w, h) / min(w, h)
    

    if aspect_ratio < 2:
    #if aspect_ratio < 3:  
      # Desenha um quadrado
      box = cv2.boxPoints(rect)
      box = np.int0(box)

      if area > 2000 and area < 6543.5:
        cv2.drawContours(img_contours, [box], 0, (0, 0, 255), 2)
        print("================================================================================================================================================================")
        print("================================================================================================================================================================") 
        print(" -> circularity (Quadrado): " + str(circularity)  + " / Area (Quadrado): " + str(area)  + " / aspect_ratio (Quadrado): " + str(aspect_ratio)  + " / radius (Quadrado): " + str(radius))
        print("================================================================================================================================================================")
        print("================================================================================================================================================================") 
        objetos = objetos + 1
    else:
        print("circularity (Quadrado): " + str(circularity)  + " / Area (Quadrado): " + str(area)  + " / aspect_ratio (Quadrado): " + str(aspect_ratio)  + " / radius (Quadrado): " + str(radius))

print("n/Foram encontrados: " + str(objetos) + " Objetos") 
cv2_imshow(img_contours)
print()
print()